In [21]:
import numpy as np
import pandas as pd

from glob import glob
from sklearn.metrics import classification_report

from sys import path
path.append("../src")

from utils import load_pickle

In [4]:
base_models = "features-idarkvec-igcngru_features"
SOURCES = ["darknet", "honeypot", "darknet-honeypot"]

In [5]:
p_dark = load_pickle(f"../data/2022/output/darknet/{base_models}/report/k3.pkl")

In [6]:
p_dark.keys(), p_dark["20221021"].keys(), p_dark["20221021"][0].keys()

(dict_keys(['20221021', '20221022', '20221023', '20221024', '20221025', '20221026', '20221027', '20221028', '20221029', '20221030', '20221031']),
 dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 dict_keys(['y', 'preds']))

In [7]:
p_dark["20221021"][0]['y'].shape

(129,)

In [22]:
days = sorted(
    [
        f.split("/")[-1].split("_")[-2]
        for f in glob(
            f"../data/2022/input/stacking_predictions/out/k3/darknet/test/idarkvec*_fold00.csv"
        )
    ]
)
days

['20221021',
 '20221022',
 '20221023',
 '20221024',
 '20221025',
 '20221026',
 '20221027',
 '20221028',
 '20221029',
 '20221030',
 '20221031']

In [43]:
f1s = []
for day in days:
    y, preds = [], []
    for fold in np.arange(10):
        ys, ps = [], []
        for source in SOURCES:
            data = load_pickle(f"../data/2022/output/{source}/{base_models}/report/k3.pkl")
            ys.append(data["20221021"][fold]['y'])
            ps.append(data["20221021"][fold]["preds"])
        y.append(np.hstack(ys))
        preds.append(np.hstack(ps))

    y = np.hstack(y)
    preds = np.hstack(preds)
    report = classification_report(y, preds, labels=np.unique(y), output_dict=True, zero_division=0, digits=4)
    df = pd.DataFrame(report).T.sort_index()["f1-score"]
    f1s.append(df.values)

In [44]:
m = np.mean(np.vstack(f1s), axis=0)
m

array([0.97297297, 1.        , 0.98447894, 0.85714286, 0.35714286,
       0.88712629, 0.98059952, 0.99443052, 0.99065421, 1.        ,
       0.9982669 , 0.73913043, 0.88888889, 0.94434251, 0.83495146,
       0.85736555, 0.97942509])

In [49]:
def truncate(value):
    return np.trunc(value * 100) / 100

pd.DataFrame({"index": df.index, "f1": m}).set_index(["index"]).apply(lambda t: truncate(t))

,f1
index,
censys,0.97
driftnet,1.00
internetcensus,0.98
intrinsec,0.85
ipip,0.35
macro avg,0.88
micro avg,0.98
mirai,0.99
onyphe,0.99
